In [1]:
import pandas as pd
import zipfile
import seaborn as sns
import numpy as np
import pickle

In [2]:
data_path="../data/data_accuracy"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/M5_forecasting

/content/drive/MyDrive/M5_forecasting


In [5]:
data_path="/content/drive/MyDrive/M5_forecasting/data/data_accuracy"

In [ ]:
sales_train = pd.read_csv(data_path+"/sales_train_validation.csv")

In [ ]:
list_levels = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

In [ ]:
import itertools
itertools.combinations(list_levels,r=2)

In [ ]:
all_combs = []
for r in range(1,4):
  all_combs.extend(list(itertools.combinations(list_levels,r=r)))

In [ ]:
all_combs=all_combs[:-1]

In [ ]:
all_combs.extend([('overall',)])

In [ ]:
final_combs=[]
for combs in all_combs:
    combs2=[]
    for level in combs:
        combs2.append(level)
    final_combs.append(combs2)

In [ ]:
overall = pd.DataFrame(sales_train[[a for a in sales_train.columns if "d_" in a]].sum())

In [ ]:
overall.columns=['value']

In [ ]:
overall.reset_index(inplace=True)

In [ ]:
overall = overall.pivot_table(values=['value'],columns=['index'])

In [ ]:
cols=[col for col in sales_train.columns if "d_" in col]

In [ ]:
overall = overall[cols]

In [ ]:
overall.reset_index(inplace=True)
overall.drop(['index'],axis=1,inplace=True)
overall.insert(0,"id","overall")
overall.insert(1,"idmap","overall")

In [ ]:
def get_grouped_series(x,comb):
    if("overall" not in comb):
      temp_df=x.groupby(comb).sum().reset_index()
      temp_df.insert(0,"id",temp_df[comb].agg('|'.join, axis=1))
      temp_df.insert(1,"idmap","|".join(comb))
      temp_df.drop(comb,axis=1,inplace=True)
      return temp_df

In [ ]:
grouped_time_series=[]

for comb in final_combs:
  grouped_time_series.append(get_grouped_series(sales_train,comb))


In [ ]:
grouped_data = pd.concat(grouped_time_series,ignore_index=True)

In [ ]:
grouped_data.shape

In [ ]:
grouped_data.drop_duplicates(cols,inplace=True)

In [ ]:
grouped_data.shape

In [ ]:
grouped_data

In [ ]:
grouped_data = grouped_data.append(overall,ignore_index=True)

In [ ]:
grouped_data.to_csv(data_path+"/grouped_data.csv")

In [ ]:
##end of file